In [161]:
#libraries
import numpy as np 
import pandas as pd

**Вариант 2. Задание 4.**

\begin{equation*}
 \begin{cases}
    \frac{\partial u}{\partial t} = \frac{\partial}{\partial x}(u^{3/2}\frac{\partial u}{\partial x}) + \frac{\partial}{\partial y}(u^{3/2}\frac{\partial u}{\partial y}) , 0 < t\le 1, 0 < x,y < 1 \\
    u(0, x, y) = \frac{(1 + x + y)^{4/3}}{100^{1/3}}, 0 \le x,y \le 1 \\
    u(t. 0, y) = \frac{(1 + y)^{4/3}}{(10 - 28t/3)^{2/3}}, 0 < t \le 1, 0 \le y \le 1\\
    u(t, 1, y) = \frac{(2 + y)^{4/3}}{(10 - 28t/3)^{2/3}}, 0 < t \le 1, 0 \le y \le 1\\
    u(t, x, 0) = \frac{(1 + x)^{4/3}}{(10 - 28t/3)^{2/3}}, 0 < t \le 1, 0 < x \le 1\\
    u(t, x, 1) = \frac{(2 + x)^{4/3}}{(10 - 28t/3)^{2/3}}, 0 < t \le 1, 0 < x \le 1\\   
 \end{cases}
\end{equation*}

In [162]:
def f_private(t, r, phi):
    '''
    функция частного решения
    '''
    return (1 + r + phi)**(4/3) / (10 - 28 * t / 3)**(2/3)

def private_solution(t, r, phi):
    '''
    частное решение на сетке
    '''
    u = np.zeros((len(phi), len(r)))
    for i in range(len(phi)):
        u[i, :] = f_private(t, r, phi[i])
    return u

In [163]:
def grid(L, M, N):
    '''
    returns np.arrays of x, t broken into L and N pieces
    '''
    return np.linspace(0, 1, L), np.linspace(0, 1, M), np.linspace(0, 1, N)#менять первый и второй параметр, если изменятся границы

def sweep_mthd(a, b, c, d, m, l, u_ex):
    '''
    Метод прогонки
    '''
    alpha = np.zeros((m, l))
    beta = np.zeros((m, l))

    x = u_ex.copy()
    
    for i in range(1, l - 1):
        alpha[1:-1,i] = (-a(i) / (c(i) * alpha[1:-1,i-1] + b(i)))
        beta[1:-1,i] = (d(i) - c(i) * beta[1:-1,i-1]) / (b(i) + c(i) * alpha[1:-1,i-1])
    
    for i in reversed(range(1, l-1)):
        x[1:-1,i] = alpha[1:-1,i] * x[1:-1,i + 1]  + beta[1:-1,i]

    return x

def accuracy(u1, u2):
    '''
    Вычислят точность
    Returns bool
    '''
    max = 0
    epsilon = 0.01#точность
    for m in range(u1.shape[0]):
        for l in range(u1.shape[1]):
            if u2[m][l] != 0:
                num = abs((u2[m][l] - u1[m][l]) / u2[m][l])
                if num > max:
                    max = num
            if max > epsilon:
                return False
    return True

def first_step(u, u1, r, m, n, mu, tau, hr):
    
    def a_l(l):
        return -(u1[1:-1, l + 1]**mu + u1[1:-1, l]**mu) * tau / (2 * hr**2)
    def c_l(l):
        return -(u1[1:-1, l]**mu + u1[1:-1, l - 1]**mu) * tau / (2 * hr**2)
    def b_l(l):    
        return 1 - a_l(l) - c_l(l)
    def d_l(l):
        return u[n, 1:- 1, l]
    
    return sweep_mthd(a_l, b_l, c_l, d_l, m, len(r), u[n+1,:,:])

def second_step(u, u_tilda, u_f, r, m, mu, tau, hf):

    u1 = u.T
    u_tilda1 = u_tilda.T

    def a_m(m):
        return -(u1[1:-1, m + 1]**mu + u1[1:-1, m]**mu) * tau / (2 * hf**2)
    def c_m(m):
        return -(u1[1:-1, m]**mu + u1[1:-1, m - 1]**mu) * tau / (2 * hf**2)
    def b_m(m):
        return 1 - a_m(m) - c_m(m)
    def d_m(m):
        return u_tilda1[1:-1, m]
    
    return sweep_mthd(a_m, b_m, c_m, d_m, len(r), m, u_f.T).T

    
    
def numerical_solve(r, phi, t, mu):
    hr = 1 / (len(r) - 1)
    hf = 1 / (len(phi) - 1)
    tau = 1 / (len(t) - 1)
    
    m = len(phi)
    
    u = np.zeros((len(t), len(phi), len(r)))
    
    for i in range(len(phi)):#начальное условие
        u[0, i, :] = f_private(0, r, phi[i])
    
    for i in range(1, len(t)):
        u[i, :, 0] = f_private(t[i], r[0], phi)#первое граничное условие
        u[i, :, -1] = f_private(t[i], r[-1], phi)#второе граничное условие
        u[i, 0, 1:-1] = f_private(t[i], r[1:-1], phi[0])#третье граничное условие
        u[i, -1, 1:-1] = f_private(t[i], r[1:-1], phi[-1])#третье граничное условие
   
    for i in range(len(t) - 1):
        u_smth = u[i].copy()
        
        while True:
            u_tilda = first_step(u, u_smth, r, m, i, mu, tau, hr)
            u_end = second_step(u_smth, u_tilda, u[i+1], r, m, mu, tau, hf)
            if accuracy(u_smth, u_end):
                u[i + 1] = u_end
                break
            else:
                u_smth = u_end
             
    return u

In [164]:
#ТО ШО МЕНЯТЬ
L = M = 6
N = 6
mu = 1.5
T = 1

#находим аналитическое решение
r_rep, phi_rep, _ = grid(6, 6, 6) 

u_private = private_solution(T, r_rep, phi_rep)


#находим численное рашение
r, phi, t = grid(L, M, N)
u_num = numerical_solve(r, phi, t, mu)

if L != 6:
    u_numeric = u_num[-1,::2, ::2]#L=21 двойки меняем на 4, L=41 меняем 4 на 8 и так далее
else:
    u_numeric = u_num[-1]

    
print('Max error:', np.max(np.abs(u_private - u_numeric))/4)



Max error: 0.3967920535593806


In [165]:
print('_____Analytical____')
f = pd.DataFrame(u_private, columns=r_rep, index=phi_rep)
f

_____Analytical____


,0.0,0.2,0.4,0.6,0.8,1.0
0.0,1.310371,1.670972,2.052256,2.452190,2.869178,3.301927
0.2,1.670972,2.052256,2.452190,2.869178,3.301927,3.749365
0.4,2.052256,2.452190,2.869178,3.301927,3.749365,4.210586
0.6,2.452190,2.869178,3.301927,3.749365,4.210586,4.684810
0.8,2.869178,3.301927,3.749365,4.210586,4.684810,5.171361
1.0,3.301927,3.749365,4.210586,4.684810,5.171361,5.669645


In [166]:
print('_____Numerical____')
f1 = pd.DataFrame(u_numeric, columns=r_rep, index=phi_rep)
f1


_____Numerical____


,0.0,0.2,0.4,0.6,0.8,1.0
0.0,1.310371,1.670972,2.052256,2.452190,2.869178,3.301927
0.2,1.670972,1.191538,1.401848,1.568356,1.714759,3.749365
0.4,2.052256,2.160975,2.528995,2.860196,3.176504,4.210586
0.6,2.452190,2.775235,3.205978,3.611825,4.009014,4.684810
0.8,2.869178,3.285009,3.742366,4.191941,4.642034,5.171361
1.0,3.301927,3.749365,4.210586,4.684810,5.171361,5.669645


In [167]:
print('_____Errors____')
fe =  pd.DataFrame(np.abs(u_private - u_numeric)/4, columns=r_rep, index=phi_rep)
fe

_____Errors____


,0.0,0.2,0.4,0.6,0.8,1.0
0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
0.2,0.0,0.215180,0.262586,0.325206,0.396792,0.0
0.4,0.0,0.072804,0.085046,0.110433,0.143215,0.0
0.6,0.0,0.023486,0.023987,0.034385,0.050393,0.0
0.8,0.0,0.004230,0.001750,0.004661,0.010694,0.0
1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
